중심(x,y),넓이(a)

# Imports

In [3]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tqdm import tqdm_notebook
from scipy import ndimage

# Configurations

In [1]:
# BDML server GPU config
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
RCNN_DIR = os.path.join(ROOT_DIR, 'Mask_RCNN')

# Import Mask RCNN
sys.path.append(RCNN_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(RCNN_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(RCNN_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(RCNN_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "imgs/clean_2_eval_augmented")

Using TensorFlow backend.


In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.7 # 0.9->0.7 로 조정
    DETECTION_MAX_INSTANCES = 200
config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        200
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [5]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
print(len(file_names), file_names[0])

126 class_229_r_+_9.jpg


# EXTRACTE FEATURES

In [7]:
def create_input_feature(r):
    class_ids = r['class_ids']
    masks = r['masks']
    rois = r['rois']
    input_features = []
    persons_masks = np.zeros_like(masks[:,:,0])
    for i in range(len(class_ids)):

        if class_ids[i] == 1: # 사람 선택,
            target = masks[:,:,i]
            box = rois[i]
            c1, c2 = ndimage.center_of_mass(target, target)
            center = c1 - 512, c2 - 512
            y1, x1, y2, x2 = box[0], box[1], box[2], box[3]
            area = ((x2 - x1) * (y2 - y1))/1024
#             print(area, center, class_names[class_ids[i]])
            input_feature = np.concatenate((center, area.reshape((1))))
            input_features.append(input_feature)
#             print(input_feature)
            
    max_input_features = 8*3*3
    if input_features == []:
        input_features = [[0],[0]]
    random.shuffle(input_features)
    input_features = np.concatenate(input_features) #하나로
    
    # PADDING

    while(True):
        if len(input_features)  >= max_input_features:
            input_features = input_features[:max_input_features] 
            break
        
        input_features = np.append(input_features, 0)
    return input_features

In [8]:
def parse_file_name(fn):
#     print(fn)
    fn_orig = fn
    fn = fn.split('.')
    fn = fn[0].split('_')
    fn.append('dummy') # 4.jpg
    label = int(fn[1]) # class_lable_....
    reverse = False
    if 'r' in fn_orig:
        reverse = True
#     print(reverse)
    return label, reverse

In [ ]:

def create_person_mask(results, show=True):
    output = []
    
    fig = plt.figure(figsize=(15,15))
    for i in range(12):
        person_mask = np.zeros_like(results[0]['masks'][:,:,0])
        r = results[i]
        ax = plt.subplot(4,3,i+1)

        for j in range(r['masks'].shape[2]):
            height = r['masks'][:,:,j].shape[0]
            width = r['masks'][:,:,j].shape[1]

            if r['class_ids'][j] == 1:
                person_mask = person_mask + r['masks'][:,:,j]
        output.append(person_mask)
        person_mask = skimage.transform.resize(person_mask, (128,128))
        ax.imshow(person_mask)
    if show:
        plt.show()
    return output

In [ ]:
# TFDataset serialize functions

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
 
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def create_tf_example(input_feature, input_label, input_file_name):
    
        input_file_name = input_file_name.encode()
        input_feature = np.array(input_feature).tostring()
        input_label = np.array(input_label).tostring()
        features = {
            'file_name':_bytes_feature(input_file_name),
            'feature':_bytes_feature(input_feature),
            'label':_bytes_feature(input_label)
        }
        features = tf.train.Features(feature=features)
        example = tf.train.Example(features=features)
        return example


In [ ]:
with open('../label/clean_2_label.pickle', mode='rb') as f:
    labels = np.array(pickle.load(f))
with open('../label/clean_2_label_r.pickle', mode='rb') as f:
    labels_r = np.array(pickle.load(f))

In [ ]:
DIR_PATH = '../input_feature'
TFRecord_file = 'clean_3_eval.TFRecord'

batches = []
input_features = []
input_labels = []
input_file_names = []
BATCH_SIZE = 1

with tf.python_io.TFRecordWriter(os.path.join(DIR_PATH, TFRecord_file)) as writer:
    for file_name in tqdm_notebook(file_names):
        input_file_names.append(file_name)

        label, reverse = parse_file_name(file_name)
        if reverse:
            input_labels.append(labels_r[label])
        else:
            input_labels.append(labels[label])

        image = skimage.io.imread(os.path.join(IMAGE_DIR, file_name))
        batches.append(image)

        if len(batches) == BATCH_SIZE:
            results = model.detect(batches, verbose=0)
    #         create_person_mask(results)
            for i in range(len(results)):
                input_feature = create_input_feature(results[i])
                input_features.append(input_feature)
    #         visualize.display_instances(batches[0], results[0]['rois'], results[0]['masks'], 
    #                                  results[0]['class_ids'], class_names, results[0]['scores'])
    #         print(input_features[0])
    #         print(input_labels[0])
    #         print(input_file_names[0])
#             visualize.display_instances(batches[0], results[0]['rois'], 
#                                         results[0]['masks'], 
#                                         results[0]['class_ids'], 
#                                         class_names)
            assert len(input_features) == BATCH_SIZE
            assert len(input_labels) == BATCH_SIZE
            assert len(input_file_names) == BATCH_SIZE

            for i in range(BATCH_SIZE):
                writer.write(create_tf_example(input_features[i],
                                               input_labels[i],
                                               input_file_names[i]).SerializeToString())
            input_features = []
            input_labels = []
            input_file_names = []
            batches = []


                    